In [10]:
import random
import exrex

def rand_blocks(len=2, regex="[a-z]{3}:\d{3}"):
    return

exrex.getone('[a-z]{5}-([a-z]|\d){3}-[A-Z]{3}-\d{3}')

'cfgzb-vm2-XFW-508'

In [ ]:
import pandas as pd
import random
import numpy as np
import geopandas as gpd
import os
import exrex

df = pd.DataFrame({"x" : [random.randint(0,100) for x in range(100)], "y":[random.randint(200,300) for y in range(100)]})

def distribute_nans(df, percent=[5,10]):
    if isinstance(percent, list): 
        nan_prop = [round(np.floor(df.size * (x/100))) for x in percent]
    else:
        nan_prop = round(np.floor(df.size * (percent/100)))

    for i, prop in enumerate(nan_prop):
        df_temp = df.iloc[:,i:i+1].sample(frac=1)
        print(df_temp.iloc[0:prop, :].apply(lambda x: np.nan, axis=1))
        print(df_temp)
    return

def distribute_nans(df, percent=[5,10]):
    if isinstance(percent, list): 
        print(df.size)
        nan_prop = [round(len(df) * (x/100)) for x in percent]
        print(nan_prop)
    else:
        nan_prop = round(len(df) * (percent/100))
        print(nan_prop)

    for i, prop in enumerate(nan_prop):       
        change = df.iloc[:,i:i+1].sample(frac=1).index[0:prop]
        df.iloc[change, i:i+1] = np.nan
        df.sort_index()
    return df

df_nan = distribute_nans(df)

print(df_nan)



200
[5, 90]
       x      y
0   14.0    NaN
1   43.0    NaN
2   98.0    NaN
3   11.0    NaN
4   73.0    NaN
..   ...    ...
95  33.0    NaN
96  46.0    NaN
97   3.0  247.0
98   6.0    NaN
99  85.0    NaN

[100 rows x 2 columns]


In [249]:
import geopandas as gpd
import os

def get_var_type(var_type):
    if var_type == 'int':
        return "INTEGER"
    elif var_type == 'str':
        return "VARCHAR"
    elif var_type == 'regex':
        return "VARCHAR"
    elif var_type == 'ts':
        return "TIMESTAMP"

def gdf_to_sql(table_name, gdf, random_var, rand_var_types, rand_var_names, extra_var, extra_var_types, extra_var_name, extra_var_dict, directory):
    
    # Opens up an SQL file based on the table name, writes to the file and closes it
    sqlFile = open(f'{directory}/SQL/{table_name}.sql', "w")
    sqlFile.write("")
    sqlFile.close()

    # Opens up the SQL file to append lines to it
    sqlFile = open(f'{directory}/SQL/{table_name}.sql', "a")

    # SQL statments to create the table as well as drop if exists the table are appended
    sqlFile.write('-- This is an automatically generated SQL table. This has been generated by the RADIAN tool (developer Mr. Paddy Gorry)\n\n')

    sqlFile.write('DROP TABLE IF EXISTS {}; \n\n'.format(table_name))

    sqlFile.write('CREATE TABLE {} ( \n'.format(table_name))
    sqlFile.write('\tpkid SERIAL PRIMARY KEY NOT NULL, \n')
    sqlFile.write("\tthegeom GEOMETRY DEFAULT ST_GeomFromText('POINT(0,51)', 4326)")

    if random_var:
        sqlFile.write(',\n')
        for count, type in enumerate(rand_var_types):
            sqlFile.write(f'\t{rand_var_names[count]} {get_var_type(rand_var_types[count])}')
            if count < len(rand_var_types)-1:
                sqlFile.write(', \n')

    if extra_var:
        for variable in extra_var_dict:
            create_query = f",\n\t{variable['name']} "
            if variable['type'] == 'str':
                create_query += 'VARCHAR'
            elif variable['type'] == 'regex':
                create_query += 'VARCHAR'
            elif variable['type'] == 'int':
                create_query += 'INTEGER'
            sqlFile.write(create_query)

    sqlFile.write('\n); \n\n')

    sqlFile.write('-- Spatial index is now created\n\n')

    # Creation of Spatial Index for the SQL file
    sqlFile.write('CREATE INDEX {}_spatial_index ON {} USING gist (thegeom); \n'.format(table_name, table_name))

    # Loop through every point in the GeoDataFrame to write an instert statement to append to the SQL file
    for row in gdf.iterrows():
        # Set 'y' value to Latitude and 'x' value to Longitude.
        print("geometry:", row[1][0])
        lat = row[1][0].y
        lon = row[1][0].x
        # Pull the randomly generated strings and ints from the dataframe
        if random_var:
            if (not extra_var):
                query = f"INSERT into {table_name} (thegeom, "
                for count, type in enumerate(rand_var_names):
                    query += f"{rand_var_names[count]}"
                    if count < len(rand_var_names)-1:
                        query += ", "
                query += f") VALUES (ST_SetSRID(ST_MakePoint({lon},{lat}), 4326), "
                for count, type in enumerate(rand_var_types):
                    # 19/11/2024 - adding in NULL functionality
                    current_value = row[1][count+1]
                    if np.isnan(current_value):
                        query += f'{current_value}'
                    else:
                        if type != "int":
                            query += "'"
                        query += f'{current_value}'
                        if type != "int":
                            query += "'"
                    if count < len(rand_var_names)-1:
                        query += ", "
                query += '); \r'           

            else:
                full_var_names = rand_var_names + extra_var_name
                full_var_types = rand_var_types + extra_var_types

                extra_values = []
                for i in range(len(extra_var_name)):
                    extra_values.append(row[1][len(rand_var_names)+i])

                query = f"INSERT into {table_name} (thegeom, "
                for count, type in enumerate(full_var_names):
                    query += f"{full_var_names[count]}"
                    if count < len(full_var_names)-1:
                        query += ", "
                query += f") VALUES (ST_SetSRID(ST_MakePoint({lon},{lat}), 4326), "
                for count, type in enumerate(full_var_types):
                    current_value = row[1][count+1]
                    if np.isnan(current_value):
                        query += f'{current_value}'
                    else:
                        if type != "int":
                            query += "'"
                    
                        if type == "str":
                            query += "{}".format(current_value.replace("'", "''")) 
                        else:
                            query += "{}".format(current_value)
                    
                        if type != "int":
                            query += "'"
                    if count < len(full_var_names)-1:
                        query += ", "
                query += '); \r'

        else:
            if(not extra_var):
                # Insert statement for each point along with the included variables
                query = "INSERT into {} (thegeom) VALUES (ST_SetSRID(ST_MakePoint({},{}),4326)); \r".format(
                    table_name, lon, lat)
            else:
                extra_values = []
                for i in range(len(extra_var_name)):
                    extra_values.append(row[1][i+1])

                var_index = 1
                query = f"INSERT into {table_name} (thegeom, "
                for count, type in enumerate(extra_var_name):
                    query += f"{extra_var_name[count]}"
                    if count < len(extra_var_name)-1:
                        query += ", "
                query += f") VALUES (ST_SetSRID(ST_MakePoint({lon},{lat}), 4326), "
                for count, type in enumerate(extra_var_types):
                    if type != "int":
                        query += "'"
                    
                    if type == "str":
                        query += "{}".format(row[1][count+1].replace("'", "''")) 
                    else:
                        query += "{}".format(row[1][count+1])
                    
                    if type != "int":
                        query += "'"
                    if count < len(extra_var_name)-1:
                        query += ", "
                query += '); \r'

        # Write query string to SQL file
        sqlFile.write(query)

directory = os.path.dirname("geojson_polygons/paris.geojson")

random_var = True
random_var_dict = [{"type":"int", "name": "housePrice", "params": [150000,2500000]},
			    {"type":"regex", "name": "postCode", "params": "[a-zA-Z]{2}\\d{5}"},
				{"type":"regex", "name": "energyRating", "params": "[A-F]"},
				{"type":"ts", "name": "surveyTime", "params": ["2022-01-01 00:00:00", "2025-12-31 23:59:59"] }]
rand_var_types = [var_dict["type"] for var_dict in random_var_dict]
rand_var_names = [var_dict["name"] for var_dict in random_var_dict]
rand_var_params = [var_dict["params"] for var_dict in random_var_dict]

extra_var = True
extra_var_dict = [{"type":"str", "name": "streetName", "source": "street_names.csv"}]
extra_var_types = [var_dict["type"] for var_dict in extra_var_dict]
extra_var_name = [var_dict["name"] for var_dict in extra_var_dict]

paris_null = gpd.read_file('/home/paddy/git/radian/geojson_polygons/GeoJSON/null_testing.geojson')
paris_null = paris_null[['PKID', 'geometry', 'housePrice', 'postCode', 'energyRating', 'surveyTime', 'streetName']]
gdf_to_sql("sql_revamp", paris_null, random_var, rand_var_types, rand_var_names, extra_var, extra_var_types, extra_var_name, extra_var_dict, directory)

print(paris_null)

geometry: 0


/home/paddy/miniconda3/envs/radian/lib/python3.11/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/tmp/ipykernel_9247/1202543704.py:61: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("geometry:", row[1][0])
/tmp/ipykernel_9247/1202543704.py:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lat = row[1][0].y


AttributeError: 'int' object has no attribute 'y'

In [ ]:
import pandas as pd
import random
import numpy as np
import geopandas as gpd
import os
import exrex

paris_null = gpd.read_file('/home/paddy/git/radian/geojson_polygons/GeoJSON/null_testing.geojson')
directory = os.path.dirname("geojson_polygons/paris.geojson")

def gdf_to_sql_new(gdf, table_name):

    ###### SQL BOILER PLATE ######
    
    # Opens up an SQL file based on the table name, writes to the file and closes it
    sqlFile = open(f'{directory}/SQL/{table_name}.sql', "w")
    sqlFile.write("")
    sqlFile.close()

    # Opens up the SQL file to append lines to it
    sqlFile = open(f'{directory}/SQL/{table_name}.sql', "a")

    # SQL statments to create the table as well as drop if exists the table are appended
    sqlFile.write('-- This is an automatically generated SQL table. This has been generated by the RADIAN tool (developer Mr. Paddy Gorry)\n\n')

    sqlFile.write('DROP TABLE IF EXISTS {}; \n\n'.format(table_name))

    sqlFile.write('CREATE TABLE {} ( \n'.format(table_name))
    sqlFile.write('\tpkid SERIAL PRIMARY KEY NOT NULL, \n')

    ###### BOILER PLATE ENDS ######

    def get_var_type(dtype):
        match dtype:
            case 'int64':
                return "INTEGER"
            case 'float64':
                return "REAL"
            case 'object':
                return "VARCHAR"
            case 'datetime64[ns]':
                return "TIMESTAMP"
            case 'geometry':
                return "GEOMETRY DEFAULT ST_GeomFromText('POINT(0,51)', 4326)"
            case _:
                return "NONE"    

    def ddl_column(col):
        return f"\t{col.name} {get_var_type(col.dtype)}"
        
    ###### DDL statements for creating variables from the gdf columns ######
    gdf.columns = ['thegeom' if x=='geometry' else x for x in gdf.columns]
    gdf_cols = [(gdf[gdf.columns[x]]) for x in range(1,len(gdf.columns))]

    # 'geometry' is not a valid column name for the SQL file, so we check and change this
    geom_col_name = [col.name for col in gdf_cols if col.dtype=='geometry'][0]

    # PKID serial has already been created
    for i, col in enumerate(gdf_cols):
        sqlFile.write("{}{}".format(ddl_column(col), (",\n" if i < len(gdf.columns)-2 else "\n);"))) #-2 since we omit the first column (pkid)

    sqlFile.write('\n\n-- Spatial index is now created\n\n')

    # Creation of Spatial Index for the SQL file
    sqlFile.write(f"CREATE INDEX {table_name}_spatial_index ON {table_name} USING gist ({geom_col_name}); \n")

    test_row = gdf.iloc[[0]]

    ###### DDL Insert Statements ######
    def write_insert(row):
        sqlFile.write(f"INSERT INTO {table_name} (")
        [sqlFile.write(f"{col}, ") if i < len(row.columns)-1 else sqlFile.write(f"{col}) VALUES (") for i, col in enumerate(row.columns) if i > 0]
        [sqlFile.write(f"{val}, ") if i < len(row.values[0])-1 else sqlFile.write(f"{val});\n") for i, val in enumerate(row.values[0]) if i > 0]


gdf_to_sql_new(paris_null, "sql_test")

test = [(paris_null[paris_null.columns[x]]) for x in range(1,len(paris_null.columns))]
test_row = paris_null.iloc[[0]]

print([val for val in test_row.values[0]])

/home/paddy/miniconda3/envs/radian/lib/python3.11/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


AttributeError: 'numpy.ndarray' object has no attribute 'columns'